In [1]:

import get_env
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment

env = get_env.get_remote_env()
t_env = StreamTableEnvironment.create(env)


conf = t_env.get_config().get_configuration()
 
# === Python Exec Location ===
conf.set_string("python.executable", "/usr/bin/python3")
conf.set_string("pipeline.jars", "file:///opt/flink/plugins/gs-fs-hadoop/flink-gs-fs-hadoop-1.20.2.jar")  # client-side path

# === Allow fallback to Hadoop FS for gs:// and s3:// ===
conf.set_string("fs.allowed-fallback-filesystems", "hadoop")
t_env.get_config().set("parallelism.default", "1")


2025-12-02T14:14:37.440079Z main ERROR Reconfiguration failed: No configuration found for '567d299b' at 'null' in 'null'
2025-12-02T14:14:38.673350Z Thread-3 ERROR Reconfiguration failed: No configuration found for '761badbc' at 'null' in 'null'


In [2]:
# --- Create an Iceberg Hadoop catalog backed by MinIO (no Hive required) ---
# Bucket 'warehouse' must already exist; subpath 'iceberg' will be created as needed

t_env.execute_sql("DROP CATALOG IF EXISTS local_ice")

t_env.execute_sql("""
CREATE CATALOG local_ice WITH (
  'type' = 'iceberg',
  'catalog-type' = 'hadoop',
  'warehouse' = 'gs://gks-datalake/warehouse',
  'property-version' = '1'
)
""")

# Use the catalog + create/use a database
t_env.execute_sql("USE CATALOG local_ice")
t_env.execute_sql("CREATE DATABASE IF NOT EXISTS db")
t_env.execute_sql("USE db")

# --- Create an Iceberg table (format v2 recommended) ---
t_env.execute_sql("""
CREATE TABLE IF NOT EXISTS hello (
  id  INT,
  txt STRING
) WITH (
  'format-version' = '2'
)
""")

# Write some data (INSERT INTO is streaming; this returns a TableResult)
t_env.execute_sql("INSERT INTO hello VALUES (1,'hi'), (2,'there')").wait()

# Read it back (TableResult prints to stdout if you collect())
# For quick demo: do a SELECT and print the first few rows
res = t_env.execute_sql("SELECT * FROM hello")
print(res.get_job_client().get_job_id())  # optional: show job id
for i, row in enumerate(res.collect()):
    print(row)
    if i >= 9:
        break


dae96f8e6620c3e0ac95ff1e657236ab
<Row(1, 'hi')>
<Row(2, 'there')>


In [3]:

# Read it back (TableResult prints to stdout if you collect())
# For quick demo: do a SELECT and print the first few rows
res = t_env.execute_sql("SELECT * FROM hello")
print(res.get_job_client().get_job_id())  # optional: show job id
for i, row in enumerate(res.collect()):
    print(row)
    if i >= 9:
        break

3876580c0dca5fae301c3aea9413a039
<Row(1, 'hi')>
<Row(2, 'there')>


In [4]:
t_env.execute_sql("SHOW CATALOGS").print()

+-----------------+
|    catalog name |
+-----------------+
| default_catalog |
|       local_ice |
+-----------------+
2 rows in set


In [5]:
t_env.execute_sql("SHOW DATABASES in local_ice").print()

+---------------+
| database name |
+---------------+
|            db |
+---------------+
1 row in set


In [7]:
t_env.execute_sql("SHOW TABLES in db").print()

+------------+
| table name |
+------------+
|      hello |
+------------+
1 row in set


In [6]:
t_env.execute_sql("select * from hello").print()

+----+-------------+--------------------------------+
| op |          id |                            txt |
+----+-------------+--------------------------------+
| +I |           1 |                             hi |
| +I |           2 |                          there |
+----+-------------+--------------------------------+
2 rows in set


In [7]:
t_env.execute_sql("select * from local_ice.db.hello").print()

+----+-------------+--------------------------------+
| op |          id |                            txt |
+----+-------------+--------------------------------+
| +I |           1 |                             hi |
| +I |           2 |                          there |
+----+-------------+--------------------------------+
2 rows in set
